In [41]:
import yfinance as yf
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
import concurrent.futures



n = 60, 2022:-18.45, 2020: 321.17 2015:599
n = 75, 2022: -10.81,2020: 244, 2015: 315.3
n = 185, 2015:7073%

In [2]:

def is_upward_trend(data, n=25, slope = 0.25):
    if len(data) < n:
        return False

    # Use the last n days of data for trend analysis
    data = data[-n:]

    # Prepare data for linear regression
    x = np.arange(n).reshape(-1, 1)
    y_highs = data['High'].values
    y_lows = data['Low'].values

    # Perform linear regression on both highs and lows
    reg_highs = LinearRegression().fit(x, y_highs)
    reg_lows = LinearRegression().fit(x, y_lows)

    # Check if both the highs and lows are trending upwards
    if reg_highs.coef_ > slope and reg_lows.coef_ > slope:
        return True

    return False

def is_high_volume(data, i, n=10, co_ef = 1):
    if i < n:
        return False

    today_volume = data.iloc[i]['Volume']
    avg_volume = data.iloc[i - n:i]['Volume'].mean()

    return today_volume > (avg_volume * co_ef)

def get_ma_month(data):
    return data['Close'].rolling(window=20).mean()

def get_ma_5(data):
    return data["Close"].rolling(window=5).mean()

def get_ma_10(data):
    return data["Close"].rolling(window=10).mean()

def calculate_kdj(stock_data, k_period=9, d_period=3, j_period=3):
    low_min = stock_data['Low'].rolling(window=k_period).min()
    high_max = stock_data['High'].rolling(window=k_period).max()
    rsv = ((stock_data['Close'] - low_min) / (high_max - low_min)) * 100

    # Initialize K, D, and J series with zeros
    k = pd.Series(index=stock_data.index, dtype=float)
    d = pd.Series(index=stock_data.index, dtype=float)
    j = pd.Series(index=stock_data.index, dtype=float)

    k.iloc[k_period - 1] = rsv.iloc[k_period - 1]
    d.iloc[k_period - 1] = rsv.iloc[k_period - 1]

    for i in range(k_period, len(stock_data)):
        k.iloc[i] = (2 / 3) * k.iloc[i - 1] + (1 / 3) * rsv.iloc[i]
        d.iloc[i] = (2 / 3) * d.iloc[i - 1] + (1 / 3) * k.iloc[i]
        j.iloc[i] = 3 * k.iloc[i] - 2 * d.iloc[i]

    # Fill NaN values with zeros
    k = k.fillna(0)
    d = d.fillna(0)
    j = j.fillna(0)

    return k, d, j


In [ ]:
def calculate_kdj(stock_data, k_period=9, d_period=3, j_period=3):
    low_min = stock_data['Low'].rolling(window=k_period).min()
    high_max = stock_data['High'].rolling(window=k_period).max()
    rsv = ((stock_data['Close'] - low_min) / (high_max - low_min)) * 100

    # Calculate K using vectorized operations
    k = rsv.ewm(alpha=1/3, adjust=False).mean()
    
    # Calculate D using vectorized operations
    d = k.ewm(alpha=1/3, adjust=False).mean()

    # Calculate J using vectorized operations
    j = 3 * k - 2 * d

    return k, d, j

# skip from here


In [ ]:
# skip this now 
"""
def main():
    stock_symbols = ['QQQ']
    """
    stock_symbols = [
        'AAPL',
        'MSFT',
        'AMZN',
        'GOOG',
        'NVDA',
        'TSLA',
        'BRK-B',
        'JPM',
        'JNJ',
        'V',
        'PG',
        'MA',
        'UNH',
        'DIS',
        'HD',
        'NFLX',
        'BAC',
        'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    ]
    """

#    stock_symbols = ['TSLA']
    initial_investment = 10000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2014-06-01'
    end='2023-03-15'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
    for symbol in stock_symbols:
        stock_data = yf.download(symbol, start=start, end=end)
#        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
        stock_data['K'], stock_data['D'], stock_data['J'] = calculate_kdj(stock_data)
        stock_data['MA_month'] = get_ma_month(stock_data)
#        stock_data['MA_5'] = get_ma_5(stock_data)
#        stock_data['MA_10'] = get_ma_10(stock_data)

        for i in range(1, len(stock_data) - 1):
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
#            print(row_today)
#            print(type(row_today))
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]

            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < 70 and row_yesterday['D'] < 70 and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D']:
                
                if is_high_volume(stock_data, i) and \
                is_upward_trend(stock_data.iloc[:i+1]) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                    # Buy the stock
                    num_shares = cash // row_today['Close']
                    if num_shares > 0:
                        if symbol not in stocks_owned.keys():
                            stocks_owned[symbol] = num_shares
                        else:
                            stocks_owned[symbol] += num_shares
                            
                        cash -= num_shares * row_today['Close']
                      
                        print(stock_data.index[i])
                    
                        print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
            if row_yesterday['Close'] > row_yesterday['MA_month']*0.95 and row_today['Close'] < row_today['MA_month'] * 0.95:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    print(stock_data.index[i])
                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

        ta = yf.download(symbol, start=start, end=end)
        print(cash)
        print(shares)
        print(ta.iloc[-1]['Close'])
        total_value += shares * ta.iloc[-1]['Close']
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")
    
"""


In [ ]:
# skip for test
"""
start='2021-06-01'
end='2023-03-15'
stock_data_full = yf.download(stock_symbols, start=start, end=end)
"""

In [ ]:
def main(kdj_thresh = 70, sell_thresh = 0.95):
#    stock_symbols = ['GOOG',"ABNB", "NVDA", "ASML", "SMH"]
    
    stock_symbols = [
        'AAPL',
        'MSFT',
        'AMZN',
        'GOOG',
        'NVDA',
        'TSLA',
        'BRK-B',
        'JPM',
        'JNJ',
        'V',
        'PG',
        'MA',
        'UNH',
        'DIS',
        'HD',
        'NFLX',
        'BAC',
       
    ]
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 10000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2014-06-01'
    end='2023-03-15'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D']:
                
                if is_high_volume(stock_data, i) and \
                is_upward_trend(stock_data.iloc[:i+1]) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                    # Buy the stock
                    num_shares = cash // row_today['Close']
                    if num_shares > 0:
                        if symbol not in stocks_owned.keys():
                            stocks_owned[symbol] = num_shares
                        else:
                            stocks_owned[symbol] += num_shares
                            
                        cash -= num_shares * row_today['Close']
                      
                        print(stock_data.index[i])
                    
                        print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    print(stock_data.index[i])
                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * ta.iloc[-1]['Close']
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")


In [ ]:
main()

In [ ]:
!pip install scikit-optimize


In [ ]:
# this is for tuning parameters

def main(params, return_only=True):
#    stock_symbols = ['GOOG',"ABNB", "NVDA", "ASML", "SMH"]
    
    stock_symbols = [
        'AAPL',
        'MSFT',
        'AMZN',
        'GOOG',
        'NVDA',
        'TSLA',
        'BRK-B',
        'JPM',
        'JNJ',
        'V',
        'PG',
        'MA',
        'UNH',
        'DIS',
        'HD',
        'NFLX',
        'BAC',
       
    ]
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 10000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2014-06-01'
    end='2023-03-15'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D']:
                
                if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
                is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                    # Buy the stock
                    num_shares = cash // row_today['Close']
                    if num_shares > 0:
                        if symbol not in stocks_owned.keys():
                            stocks_owned[symbol] = num_shares
                        else:
                            stocks_owned[symbol] += num_shares
                            
                        cash -= num_shares * row_today['Close']
                      
#                        print(stock_data.index[i])
                    
#                        print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
#                    print(stock_data.index[i])
#                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
#                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * ta.iloc[-1]['Close']
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
#    print(f"\nInitial investment: ${initial_investment:.2f}")
#    print(f"Total value: ${total_value:.2f}")
#    print(f"Return rate: {return_rate:.2f}%")
    if return_only:
        return -return_rate
    else:
        # Print results as before
        print(f"\nInitial investment: ${initial_investment:.2f}")
        print(f"Total value: ${total_value:.2f}")
        print(f"Return rate: {return_rate:.2f}%")





In [ ]:
from skopt import gp_minimize

def tune_parameters():
    search_space = [
        (5, 180),  # n_upward_trend
        (-0.5, 0.5),  # slope_upward_trend
        (3, 60),  # n_high_volume
        (0.5, 2),  # co_ef_high_volume
        (30, 100),  # kdj_thresh
        (0.6, 1),  # sell_thresh
    ]

    result = gp_minimize(main, search_space, n_calls=50, random_state=42)

    best_params = result.x
    highest_return_rate = -result.fun

    print("\nBest parameters found:")
    print(f"n_upward_trend: {best_params[0]}, slope_upward_trend: {best_params[1]}")
    print(f"n_high_volume: {best_params[2]}, co_ef_high_volume: {best_params[3]}")
    print(f"kdj_thresh_main: {best_params[4]}, sell_thresh_main: {best_params[5]}")
    print(f"Highest return rate: {highest_return_rate:.2f}%")

tune_parameters()


# best parameters
Best parameters found:
n_upward_trend: 125, slope_upward_trend: -0.094420236695826
n_high_volume: 6, co_ef_high_volume: 1.9080519595723013
kdj_thresh_main: 70, sell_thresh_main: 0.6943040842662802
Highest return rate: 542.43%

# start here

#### generate sp500 list

In [3]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_list = list(sp500[0]['Symbol'])
sp500_list.remove("BRK.B")
sp500_list.remove("BF.B")
sp500_list.append("BRK-B")
sp500_list.append("BF-B")


In [ ]:
"""
#    stock_symbols = ['QQQ',"^IXIC"]
      
    stock_symbols = [
        'AAPL',
        'MSFT',
        'AMZN',
        'GOOG',
        'NVDA',
        'TSLA',
        'BRK-B',
        'JPM',
        'JNJ',
        'V',
        'PG',
        'MA',
        'UNH',
        'DIS',
        'HD',
        'NFLX',
        'BAC',
       
    ]
"""

In [ ]:
# real function here

def main(params, return_only=True):

   
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh, stock_symbols = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 10000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2022-06-01'
    end='2023-03-23'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D']:
                
                if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
                is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                    # Buy the stock
                    num_shares = cash // row_today['Close']
                    if num_shares > 0:
                        if symbol not in stocks_owned.keys():
                            stocks_owned[symbol] = num_shares
                        else:
                            stocks_owned[symbol] += num_shares
                            
                        cash -= num_shares * row_today['Close']
                      
                        print(stock_data.index[i])
                    
                        print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    print(stock_data.index[i])
                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

    print("Buy signals:")
    print(buy_signals_df)

    print("\nSell signals:")
    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * ta.iloc[-1]['Close']
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")
    
    """
    if return_only:
        return -return_rate
    else:
        # Print results as before
        print(f"\nInitial investment: ${initial_investment:.2f}")
        print(f"Total value: ${total_value:.2f}")
        print(f"Return rate: {return_rate:.2f}%")
    """




## Now we are improving the algorithm to hold up to maximum of 15 stocks. 


In [ ]:
# real function here

def main(params, return_only=True):

   
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh, stock_symbols, n_maximum_stock = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 100000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2019-06-01'
    end='2023-03-23'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_5')] = get_ma_5(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_10')] = get_ma_10(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D'] and \
            row_today["MA_5"] > row_today["MA_10"] and row_today["MA_10"] > row_today["MA_month"]:
                
                if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
                is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})
                    if len(stocks_owned) < n_maximum_stock:
                        # Buy the stock
                        allocated_cash = cash / (n_maximum_stock - len(stocks_owned)) # Allocate cash evenly to the remaining stock positions
                        num_shares = allocated_cash // row_today['Close']
                        if num_shares > 0:
                            if symbol not in stocks_owned.keys():
                                stocks_owned[symbol] = num_shares
                            else:
                                stocks_owned[symbol] += num_shares

                            cash -= num_shares * row_today['Close']

                            print(stock_data.index[i])

                            print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    print(stock_data.index[i])
                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

    print("Buy signals:")
    print(buy_signals_df)

    print("\nSell signals:")
    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

#        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * stock_data_full.loc[stock_data_full.index[-1], (symbol, 'Close')]
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")
    
    """
    if return_only:
        return -return_rate
    else:
        # Print results as before
        print(f"\nInitial investment: ${initial_investment:.2f}")
        print(f"Total value: ${total_value:.2f}")
        print(f"Return rate: {return_rate:.2f}%")
    """




# again, test the best parameter

In [ ]:
# real function here

def main(params, return_only=True, stock_symbols = sp500_list):

   
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh, n_maximum_stock = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 100000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2019-06-01'
    end='2023-03-23'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_5')] = get_ma_5(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_10')] = get_ma_10(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D'] and \
            row_today["MA_5"] > row_today["MA_10"] and row_today["MA_10"] > row_today["MA_month"]:
                
                if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
                is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})
                    if len(stocks_owned) < n_maximum_stock:
                        # Buy the stock
                        allocated_cash = cash / (n_maximum_stock - len(stocks_owned)) # Allocate cash evenly to the remaining stock positions
                        num_shares = allocated_cash // row_today['Close']
                        if num_shares > 0:
                            if symbol not in stocks_owned.keys():
                                stocks_owned[symbol] = num_shares
                            else:
                                stocks_owned[symbol] += num_shares

                            cash -= num_shares * row_today['Close']

#                            print(stock_data.index[i])

#                            print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
#                    print(stock_data.index[i])
#                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
#                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

#        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * stock_data_full.loc[stock_data_full.index[-1], (symbol, 'Close')]
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
#    print(f"\nInitial investment: ${initial_investment:.2f}")
#    print(f"Total value: ${total_value:.2f}")
#    print(f"Return rate: {return_rate:.2f}%")
    
    
    if return_only:
        return -return_rate
    else:
        # Print results as before
        print(f"\nInitial investment: ${initial_investment:.2f}")
        print(f"Total value: ${total_value:.2f}")
        print(f"Return rate: {return_rate:.2f}%")
    




In [ ]:
from skopt import gp_minimize

def tune_parameters():
    search_space = [
        (5, 180),  # n_upward_trend
        (-1, 1),  # slope_upward_trend
        (3, 60),  # n_high_volume
        (0.5, 2),  # co_ef_high_volume
        (30, 100),  # kdj_thresh
        (0.6, 1),  # sell_thresh
        (1, 15) # n_maximum_stock
    ]

    result = gp_minimize(main, search_space, n_calls=50, random_state=42, n_jobs = -1)

    best_params = result.x
    highest_return_rate = -result.fun

    print("\nBest parameters found:")
    print(f"n_upward_trend: {best_params[0]}, slope_upward_trend: {best_params[1]}")
    print(f"n_high_volume: {best_params[2]}, co_ef_high_volume: {best_params[3]}")
    print(f"kdj_thresh_main: {best_params[4]}, sell_thresh_main: {best_params[5]}")
    print(f"n_maximum_stock: {best_params[6]}")
    print(f"Highest return rate: {highest_return_rate:.2f}%")

tune_parameters()

## best parameters
Best parameters found:
n_upward_trend: 5, slope_upward_trend: 0
n_high_volume: 3, co_ef_high_volume: 1.7247925349436912
kdj_thresh_main: 59, sell_thresh_main: 0.9357612919167262
n_maximum_stock: 3
Highest return rate: 75.08%


### current algorithm

In [54]:
# real function here

def main(params, stock_symbols = sp500_list):

   
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh, n_maximum_stock = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 100000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2015-01-01'
    end='2023-03-23'
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
    for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_5')] = get_ma_5(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_10')] = get_ma_10(stock_data_full[symbol])

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    for i in range(1, len(stock_data_full) - 1):
            
        for symbol in stock_symbols:
#            stock_data = stock_data.xs(symbol, level=1, axis=1) # get the data with that symbol
            stock_data = stock_data_full[symbol]
    #        stock_data = yf.download(symbol, start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))
            row_today = stock_data.iloc[i]
            row_yesterday = stock_data.iloc[i - 1]
            nasdaq_today = nasdaq_data.loc[row_today.name]
            nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]
            
    #        stock_data['MA_5'] = get_ma_5(stock_data)
    #        stock_data['MA_10'] = get_ma_10(stock_data)


            # Check for buy signal
#            if row_today['Close'] > row_today["MA_5"] \
            if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
            and row_today['K'] > row_today['D'] and \
            row_today["MA_5"] > row_today["MA_10"] and row_today["MA_10"] > row_today["MA_month"]:
                
                if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
                is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
                is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                    buy_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})
                    if len(stocks_owned) < n_maximum_stock:
                        # Buy the stock
                        allocated_cash = cash / (n_maximum_stock - len(stocks_owned)) # Allocate cash evenly to the remaining stock positions
                        num_shares = allocated_cash // row_today['Close']
                        if num_shares > 0:
                            if symbol not in stocks_owned.keys():
                                stocks_owned[symbol] = num_shares
                            else:
                                stocks_owned[symbol] += num_shares

                            cash -= num_shares * row_today['Close']

                            print(stock_data.index[i])

                            print("buy {}stock at:{}".format(symbol,row_today['Close']))
                
                """
                if is_high_volume(stock_data, i):
                    print(symbol)
                    print(stock_data.index[i])
                    print(stock_data.iloc[:i+1])
                    if is_upward_trend(stock_data.iloc[:i+1]):
                        
                        print("upward trend checked yes")
                        if is_upward_trend(nasdaq_data.loc[:nasdaq_today.name]):                    
                            print("nasdaq upward trend checked yes")
                
                """
            # Check for sell signal
      
            if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
                sell_signals.append({'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']})

                # Sell the stock
                if symbol in stocks_owned:
                    cash += stocks_owned[symbol] * row_today['Close']
                    start_date = datetime.strptime(start, '%Y-%m-%d')
                    print(stock_data.index[i])
                    print("sell {} stock at:{}".format(symbol, row_today['Close']))
#                    print("remaining cash is{}".format(cash))
                    del stocks_owned[symbol]

    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

#        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * stock_data_full.loc[stock_data_full.index[-1], (symbol, 'Close')]
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
#    print(f"\nInitial investment: ${initial_investment:.2f}")
#    print(f"Total value: ${total_value:.2f}")
#    print(f"Return rate: {return_rate:.2f}%")

    
    
#    if return_only:
#        return -return_rate
#    else:
        # Print results as before
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")
    return buy_signals_df, sell_signals_df
    




In [55]:
start_time = time.time()
params_ = [5,  0, 3,  1.7247925349436912, 59,  0.9357612919167262,3]
df1, df2 = main(params_)
end_time = time.time()
print(end_time - start_time)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  503 of 503 completed
2015-02-04 00:00:00
buy WHRstock at:214.8800048828125
2015-02-09 00:00:00
buy PAYCstock at:29.760000228881836
2015-02-10 00:00:00
buy MOHstock at:56.2599983215332
2015-03-11 00:00:00
sell WHR stock at:196.6699981689453
2015-03-16 00:00:00
buy BSXstock at:17.239999771118164
2015-04-29 00:00:00
sell MOH stock at:59.66999816894531
2015-05-15 00:00:00
buy YUMstock at:67.54852294921875
2015-07-02 00:00:00
sell PAYC stock at:34.029998779296875
2015-07-22 00:00:00
buy IDXXstock at:70.36000061035156
2015-08-12 00:00:00
sell YUM stock at:58.00143814086914
2015-08-24 00:00:00
sell BSX stock at:15.850000381469727
2015-08-25 00:00:00
sell IDXX stock at:69.94000244140625
2015-09-15 00:00:00
buy TERstock at:18.969999313354492
2015-09-16 00:00:00
buy STZstock at:131.16000366210938
2015-09-16 00:00:00
buy RLstock at:114.5999984741211
2015-12-11 00:00:00
sell RL 

# now i want to further speed up the algorithm by using parrelism.

In [48]:
# real function here

def main(params, stock_symbols = sp500_list):

   
    
    n_upward_trend, slope_upward_trend, n_high_volume, co_ef_high_volume, kdj_thresh, sell_thresh, n_maximum_stock = params
    
    """
     'INTC',
        'CMCSA',
        'VZ',
        'T',
        'CSCO',
        'PFE',
        'WMT',
        'XOM',
        'KO',
        'PEP',
        'MRK',
    """



    initial_investment = 100000
    cash = initial_investment
    stocks_owned = {}

    buy_signals = []
    sell_signals = []
    
    start='2019-06-01'
    end='2023-03-23'
    stock_symbols = sp500_list
    nasdaq_data = yf.download('^IXIC', start=start, end=end)
    stock_data_full = yf.download(stock_symbols, start=start, end=end)
    stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)

    # takes 48s to run
    def process_stock_symbol(symbol):
        k, d, j = calculate_kdj(stock_data_full[symbol])
        ma_month = get_ma_month(stock_data_full[symbol])
        ma_5 = get_ma_5(stock_data_full[symbol])
        ma_10 = get_ma_10(stock_data_full[symbol])
        return symbol, k, d, j, ma_month, ma_5, ma_10

    # Parallelize the loop
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(process_stock_symbol, stock_symbols)

    # Assign the results to stock_data_full
    for result in results:
        symbol, k, d, j, ma_month, ma_5, ma_10 = result
        stock_data_full[(symbol, 'K')] = k
        stock_data_full[(symbol, 'D')] = d
        stock_data_full[(symbol, 'J')] = j
        stock_data_full[(symbol, 'MA_month')] = ma_month
        stock_data_full[(symbol, 'MA_5')] = ma_5
        stock_data_full[(symbol, 'MA_10')] = ma_10

#    nasdaq_data = yf.download('^IXIC', start='2022-01-01', end=dt.datetime.now().strftime('%Y-%m-%d'))

    def process_symbol(symbol, i, nasdaq_data):
        stock_data = stock_data_full[symbol]
        row_today = stock_data.iloc[i]
        row_yesterday = stock_data.iloc[i - 1]
        nasdaq_today = nasdaq_data.loc[row_today.name]
        nasdaq_yesterday = nasdaq_data.loc[row_yesterday.name]

        buy_signal = None
        sell_signal = None

        # Check for buy signal
        if row_yesterday['K'] < kdj_thresh and row_yesterday['D'] < kdj_thresh and row_yesterday['K'] < row_yesterday['D'] \
        and row_today['K'] > row_today['D'] and \
        row_today["MA_5"] > row_today["MA_10"] and row_today["MA_10"] > row_today["MA_month"]:

            if is_high_volume(stock_data, i, n = n_high_volume, co_ef = co_ef_high_volume) and \
            is_upward_trend(stock_data.iloc[:i+1], n = n_upward_trend, slope = slope_upward_trend) and \
            is_upward_trend(nasdaq_data.loc[:nasdaq_today.name], n = n_upward_trend, slope = slope_upward_trend):                    
                buy_signal = {'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']}

        # Check for sell signal
        if row_yesterday['Close'] > row_yesterday['MA_month'] * sell_thresh \
        and row_today['Close'] < row_today['MA_month'] * sell_thresh:
            sell_signal = {'symbol': symbol, 'date': row_today.name, 'price': row_today['Close']}

        return buy_signal, sell_signal

    for i in range(1, len(stock_data_full) - 1):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(process_symbol, stock_symbols, [i]*len(stock_symbols), [nasdaq_data]*len(stock_symbols)))

            for buy_signal, sell_signal in results:
                if buy_signal:
                    symbol = buy_signal['symbol']
                    row_today = stock_data_full[symbol].iloc[i]

                    if len(stocks_owned) < n_maximum_stock:
                        # Buy the stock
                        allocated_cash = cash / (n_maximum_stock - len(stocks_owned)) # Allocate cash evenly to the remaining stock positions
                        num_shares = allocated_cash // row_today['Close']
                        if num_shares > 0:
                            if symbol not in stocks_owned.keys():
                                stocks_owned[symbol] = num_shares
                            else:
                                stocks_owned[symbol] += num_shares

                            cash -= num_shares * row_today['Close']

                            print(stock_data_full[symbol].index[i])
                            print("buy {}stock at:{}".format(symbol,row_today['Close']))
                    buy_signals.append(buy_signal)

                if sell_signal:
                    symbol = sell_signal['symbol']
                    row_today = stock_data_full[symbol].iloc[i]

                    # Sell the stock
                    if symbol in stocks_owned:
                        cash += stocks_owned[symbol] * row_today['Close']
                        start_date = datetime.strptime(start, '%Y-%m-%d')
                        print(stock_data_full[symbol].index[i])
                        print("sell {} stock at:{}".format(symbol, row_today['Close']))
                        del stocks_owned[symbol]
                    sell_signals.append(sell_signal)
                    
    buy_signals_df = pd.DataFrame(buy_signals)
    sell_signals_df = pd.DataFrame(sell_signals)

#    print("Buy signals:")
#    print(buy_signals_df)

#    print("\nSell signals:")
#    print(sell_signals_df)

    # Calculate the total value of stocks and cash
    total_value = cash
    for symbol, shares in stocks_owned.items():

#        ta = yf.download(symbol, start=start, end=end)

        total_value += shares * stock_data_full.loc[stock_data_full.index[-1], (symbol, 'Close')]
#        print(total_value)

    # Calculate the return rate
    return_rate = (total_value - initial_investment) / initial_investment * 100
#    print(f"\nInitial investment: ${initial_investment:.2f}")
#    print(f"Total value: ${total_value:.2f}")
#    print(f"Return rate: {return_rate:.2f}%")

    
    
#    if return_only:
#        return -return_rate
#    else:
        # Print results as before
    print(f"\nInitial investment: ${initial_investment:.2f}")
    print(f"Total value: ${total_value:.2f}")
    print(f"Return rate: {return_rate:.2f}%")
    return buy_signals_df, sell_signals_df
    




In [49]:
start_time = time.time()
params_ = [5,  0, 3,  1.7247925349436912, 59,  0.9357612919167262,3]
df1, df2 = main(params_)
end_time = time.time()
print(end_time - start_time)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  503 of 503 completed


C:\Users\26319\AppData\Local\Temp/ipykernel_11728/1144764796.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_data_full[(symbol, 'J')] = j
C:\Users\26319\AppData\Local\Temp/ipykernel_11728/1144764796.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_data_full[(symbol, 'MA_month')] = ma_month
C:\Users\26319\AppData\Local\Temp/ipykernel_11728/1144764796.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newfram

2019-07-12 00:00:00
buy UPSstock at:105.91000366210938
2019-07-16 00:00:00
buy SCHWstock at:41.630001068115234
2019-07-23 00:00:00
buy HASstock at:119.30999755859375
2019-08-05 00:00:00
sell SCHW stock at:38.84000015258789
2019-08-23 00:00:00
sell HAS stock at:104.12999725341797
2019-09-03 00:00:00
buy PODDstock at:167.9499969482422
2019-09-05 00:00:00
buy NVDAstock at:44.935001373291016


KeyboardInterrupt: 

(Date
 2019-06-03          NaN
 2019-06-04          NaN
 2019-06-05          NaN
 2019-06-06          NaN
 2019-06-07          NaN
                 ...    
 2023-03-16    56.624158
 2023-03-17    65.068234
 2023-03-20    72.596883
 2023-03-21    81.022842
 2023-03-22    76.227749
 Length: 959, dtype: float64,
 Date
 2019-06-03          NaN
 2019-06-04          NaN
 2019-06-05          NaN
 2019-06-06          NaN
 2019-06-07          NaN
                 ...    
 2023-03-16    43.649166
 2023-03-17    50.788855
 2023-03-20    58.058198
 2023-03-21    65.713079
 2023-03-22    69.217969
 Length: 959, dtype: float64,
 Date
 2019-06-03           NaN
 2019-06-04           NaN
 2019-06-05           NaN
 2019-06-06           NaN
 2019-06-07           NaN
                  ...    
 2023-03-16     82.574143
 2023-03-17     93.626992
 2023-03-20    101.674253
 2023-03-21    111.642368
 2023-03-22     90.247308
 Length: 959, dtype: float64)

In [36]:
import time
start_time = time.time()

start='2019-06-01'
end='2023-03-23'
stock_symbols = sp500_list
nasdaq_data = yf.download('^IXIC', start=start, end=end)
stock_data_full = yf.download(stock_symbols, start=start, end=end)
stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)

# takes 48s to run
def process_stock_symbol(symbol):
    k, d, j = calculate_kdj(stock_data_full[symbol])
    ma_month = get_ma_month(stock_data_full[symbol])
    ma_5 = get_ma_5(stock_data_full[symbol])
    ma_10 = get_ma_10(stock_data_full[symbol])
    return symbol, k, d, j, ma_month, ma_5, ma_10

# Parallelize the loop
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_stock_symbol, stock_symbols)

# Assign the results to stock_data_full
for result in results:
    symbol, k, d, j, ma_month, ma_5, ma_10 = result
    stock_data_full[(symbol, 'K')] = k
    stock_data_full[(symbol, 'D')] = d
    stock_data_full[(symbol, 'J')] = j
    stock_data_full[(symbol, 'MA_month')] = ma_month
    stock_data_full[(symbol, 'MA_5')] = ma_5
    stock_data_full[(symbol, 'MA_10')] = ma_10

#takes 149s to run

end_time = time.time()
print(end_time - start_time)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  503 of 503 completed


C:\Users\26319\AppData\Local\Temp/ipykernel_11728/4180073435.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_data_full[(symbol, 'J')] = j
C:\Users\26319\AppData\Local\Temp/ipykernel_11728/4180073435.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_data_full[(symbol, 'MA_month')] = ma_month
C:\Users\26319\AppData\Local\Temp/ipykernel_11728/4180073435.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newfram

42.42022156715393


In [38]:
stock_data_full["A","K"]

Date
2019-06-03          NaN
2019-06-04          NaN
2019-06-05          NaN
2019-06-06          NaN
2019-06-07          NaN
                ...    
2023-03-16    28.039489
2023-03-17    21.543473
2023-03-20    22.321484
2023-03-21    31.241520
2023-03-22    27.719558
Name: (A, K), Length: 959, dtype: float64

In [33]:
stock_data_full = yf.download(stock_symbols, start=start, end=end)
stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)

for symbol in stock_symbols:
        stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_5')] = get_ma_5(stock_data_full[symbol])
        stock_data_full[(symbol, 'MA_10')] = get_ma_10(stock_data_full[symbol])

[*********************100%***********************]  503 of 503 completed


In [28]:
stock_data_full[("AAL", 'K')]

KeyError: 39

In [25]:
stock_data_full

ValueError: Length of values (3038) does not match length of index (3030)

A 
 AAL 
 ... 
 ARE 
 ADM 
 AOS 
 ABT 
 ADM 
 ABT 
 A 
 ADM 
 ADBE 
 A 
 
 
 
 Adj Close 
 Close 
 High 
 Low 
 Open 
 Volume 
 Adj Close 
 Close 
 High 
 Low 
 ... 
 K 
 K 
 J 
 K 
 D 
 D 
 D 
 J 
 J 
 J 
 
 
 Date 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019-06-03 
 65.238770 
 66.989998 
 68.099998 
 66.820000 
 67.040001 
 2560600 
 26.917614 
 27.200001 
 27.809999 
 27.020000 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2019-06-04 
 66.173683 
 67.949997 
 68.000000 
 66.410004 
 67.660004 
 4402800 
 28.817678 
 29.120001 
 29.309999 
 27.500000 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2019-06-05 
 66.563225 
 68.349998 
 69.010002 
 67.889999 
 68.610001 
 2289200 
 30.044806 
 30.360001 
 30.450001 
 29.459999 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2019-06-06 
 67.352051 
 69.160004 
 69.680000 
 68.519997 
 68.519997 
 2469000 
 30.064596 
 30.379999 
 30.510000 
 29.879999 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2019-06-07 
 67.702637 
 69.519997 
 70.220001 
 69.330002 
 69.699997 
 1547400 
 30.598991 
 30.920000 
 31.150000 
 30.430000 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 2023-03-16 
 136.820007 
 136.820007 
 137.050003 
 132.369995 
 133.970001 
 1789300 
 14.120000 
 14.120000 
 14.230000 
 13.630000 
 ... 
 74.332428 
 24.220809 
 24.758720 
 46.476150 
 55.761576 
 90.911616 
 28.039489 
 27.675949 
 69.755455 
 8.963823 
 
 
 2023-03-17 
 133.179993 
 133.179993 
 136.919998 
 132.729996 
 136.889999 
 2046000 
 13.980000 
 13.980000 
 14.170000 
 13.670000 
 ... 
 97.363253 
 10.336332 
 7.176429 
 53.745736 
 68.284908 
 92.236849 
 21.543473 
 19.723934 
 73.345720 
 8.823153 
 
 
 2023-03-20 
 134.539993 
 134.539993 
 135.380005 
 132.729996 
 133.139999 
 1704500 
 13.960000 
 13.960000 
 14.210000 
 13.870000 
 ... 
 108.490897 
 15.887405 
 31.924858 
 61.566473 
 77.207948 
 104.919507 
 22.321484 
 28.537903 
 81.479878 
 10.116871 
 
 
 2023-03-21 
 137.009995 
 137.009995 
 137.419998 
 135.220001 
 135.559998 
 1392300 
 14.370000 
 14.370000 
 14.540000 
 14.250000 
 ... 
 112.597874 
 38.845516 
 53.757083 
 68.856673 
 83.437073 
 109.432813 
 31.241520 
 38.378422 
 86.762670 
 12.304263 
 
 
 2023-03-22 
 133.729996 
 133.729996 
 137.539993 
 133.600006 
 136.960007 
 1268500 
 13.820000 
 13.820000 
 14.430000 
 13.790000 
 ... 
 93.278591 
 28.092939 
 62.087423 
 72.345519 
 79.323210 
 84.730336 
 27.719558 
 44.145519 
 79.414486 
 8.401535 
 
 
 
 959 rows × 3030 columns

In [14]:
end

1679648152.8190234

In [ ]:
stock_symbols = sp500_list
start='2019-06-01'
end='2023-03-23'
nasdaq_data = yf.download('^IXIC', start=start, end=end)
stock_data_full = yf.download(stock_symbols, start=start, end=end)
stock_data_full = stock_data_full.swaplevel(axis=1).sort_index(axis=1)
for symbol in stock_symbols:
    stock_data_full[(symbol, 'K')], stock_data_full[(symbol, 'D')], stock_data_full[(symbol, 'J')] = calculate_kdj(stock_data_full[symbol])
    stock_data_full[(symbol, 'MA_month')] = get_ma_month(stock_data_full[symbol])
    stock_data_full[(symbol, 'MA_5')] = get_ma_5(stock_data_full[symbol])
    stock_data_full[(symbol, 'MA_10')] = get_ma_10(stock_data_full[symbol])

In [ ]:
# best parameter
"""params = (
    125,     # n_upward_trend
    -0.094420236695826,  # slope_upward_trend
    6,       # n_high_volume
    1.9080519595723013,  # co_ef_high_volume
    70,      # kdj_thresh
    0.6943040842662802, # sell_thresh
    sp500_list, # the sp500 list
    3 # maximum number of stock in my portfolio
)"

In [ ]:
params = (
    125,     # n_upward_trend
    -0.094420236695826,  # slope_upward_trend
    6,       # n_high_volume
    1.9080519595723013,  # co_ef_high_volume
    70,      # kdj_thresh
    0.9, # sell_thresh
    sp500_list, # the sp500 list
    15 # maximum number of stock in my portfolio
)

In [ ]:
main(params)